# Class II

Last time:

- Purity
- I/O

The remaining three classes:

- Build + test a psuedo-random number generator (PRNG)
- Build + test a [stream](https://en.wikipedia.org/wiki/Stream_%28computing%29) data structure
- Integrate with Elixir's [StreamData](https://hex.pm/packages/stream_data)

Jupyter:

- Lots of good
- Exceptions are printed as a structure (different from IEx) & stack-trace missing


---

[Linear Congruential Generator](https://en.wikipedia.org/wiki/Linear_congruential_generator)

In [5]:
m = 9; a = 2; c = 0
seed = 1

1

In [7]:
seed = Integer.mod(a * seed + c, m)

2

In [8]:
seed = Integer.mod(a * seed + c, m)

4

## Version I

In [9]:
defmodule Generator.LCG do
  def f(seed) do
    m = 9; a = 2; c = 0
    seed = Integer.mod(a * seed + c, m)
  end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 4, 192, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 152, 0, 0, 0, 17, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 1}}

In [10]:
Generator.LCG.f(Generator.LCG.f(Generator.LCG.f(1)))

8

## Aside: Function Literals

In [11]:
9

9

In [12]:
"I'm a string!"

"I'm a string!"

In [14]:
fu = fn ->
  "I'm inside a function!"
end

#Function<20.99386804/0 in :erl_eval.expr/5>

In [15]:
fu.()

"I'm inside a function!"

In [16]:
identity = fn (x) ->
  x
end

#Function<6.99386804/1 in :erl_eval.expr/5>

In [17]:
compare = fn x, y ->
  cond do
    x < y ->
      -1
    x == y ->
      0
    x > y ->
      +1
  end
end

#Function<12.99386804/2 in :erl_eval.expr/5>

In [18]:
apply(compare, [1,2])

-1

In [19]:
compare.(1,2)

-1

---

## Version II

In [20]:
defmodule Generator.LCG do
  def f(m, a, c) do
    fn (seed) ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 5, 16, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 164, 0, 0, 0, 18, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 3}}

In [21]:
m = 9; a = 2; c = 0

0

In [24]:
g = Generator.LCG.f(m, a, c)

#Function<0.64772075/1 in Generator.LCG.f/3>

In [25]:
g.(g.(g.(1)))

8

In [26]:
1
|> g.()


2

How about some documentation?

## Version III

In [27]:
modulus = 9; multiplier = 2; increment = 0;
h = Generator.LCG.f(modulus, multiplier, increment)

#Function<0.64772075/1 in Generator.LCG.f/3>

In [28]:
h.(1)

2

In [30]:
defmodule Generator.LCG do
  defstruct [
     :modulus,
     :multiplier,
     :increment
  ]

  def f(p) do
    m = p.modulus
    a = p.multiplier
    c = p.increment
    fn (seed) ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 8, 116, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 13, 0, 0, 0, 29, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 1}}

In [31]:
%Generator.LCG{}

%Generator.LCG{increment: nil, modulus: nil, multiplier: nil}

In [32]:
p = %Generator.LCG{increment: 0, modulus: 9, multiplier: 2}
h = Generator.LCG.f(p)
h.(h.(h.(1)))

8

In [37]:
defmodule Generator.LCG do
  defstruct [
     modulus:    9,
     multiplier: 2,
     increment:  0
  ]

  def f(p) do
    m = p.modulus
    a = p.multiplier
    c = p.increment
    fn (seed) ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 8, 116, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 13, 0, 0, 0, 29, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 1}}

In [38]:
%Generator.LCG{}

%Generator.LCG{increment: 0, modulus: 9, multiplier: 2}

In [40]:
p = %Generator.LCG{}
h = Generator.LCG.f(p)
h.(h.(h.(1)))

8

## Version IV

In [51]:
defmodule Generator.LCG do
  defmodule Parameters do
    defstruct [
       modulus:    9,
       multiplier: 2,
       increment:  0
    ]

    def okay?(p) do
      m = p.modulus
      a = p.multiplier
      c = p.increment
      
      x = 0 < m
      y = (0 < a and a < m)
      z = (0 <= c and c < m)

      x and y and z
    end
  end

  def f(p) do
    m = p.modulus
    a = p.multiplier
    c = p.increment
    fn (seed) ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 6, 96, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 206, 0, 0, 0, 23, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:f, 1}}

In [43]:
p = %Generator.LCG.Parameters{increment: 0, modulus: 9, multiplier: 2}
h = Generator.LCG.f(p)
h.(h.(h.(1)))

8

## Aside: Module Nesting & Name-spacing

In [49]:
defmodule M do
  defmodule N do
    def f do
      "I'm in M.N!"
    end
  end
  def f do
    N.f()
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 4, 0, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 128, 0, 0, 0, 14, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 0}}

In [50]:
M.f()

"I'm in M.N!"

---

## Version V

In [54]:
Generator.LCG.Parameters.okay?(p)

true

In [57]:
defmodule Generator.LCG do
  defmodule Parameters do
    defstruct [
       modulus:    9,
       multiplier: 2,
       increment:  0
    ]

    def okay?(p) do
      m = p.modulus
      a = p.multiplier
      c = p.increment
      
      x = 0 < m
      y = (0 < a and a < m)
      z = (0 <= c and c < m)

      x and y and z
    end
  end

  def instance(p) do
    m = p.modulus
    a = p.multiplier
    c = p.increment
    fn (seed) when seed >= 0 and seed < m ->
      Integer.mod(a * seed + c, m)
    end
 end
end

{:module, Generator.LCG, <<70, 79, 82, 49, 0, 0, 6, 224, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 236, 0, 0, 0, 24, 20, 69, 108, 105, 120, 105, 114, 46, 71, 101, 110, 101, 114, 97, 116, 111, 114, 46, 76, 67, 71, 8, 95, ...>>, {:instance, 1}}

In [58]:
x = %Generator.LCG.Parameters{
  modulus:    9,
  multiplier: 2,
  increment:  0
}
true = Generator.LCG.Parameters.okay?(x)
f = Generator.LCG.instance(x)

#Function<0.18101733/1 in Generator.LCG.instance/1>

In [59]:
f.(1)

2

In [60]:
f.(-1)

FunctionClauseError: 1

## Aside: Guard Clauses

---

In [64]:
try do
  f.(16)
rescue
  BadArg ->
    nil
end

FunctionClauseError: 1

In [62]:
defmodule M do
  def f(_) do
    nil
  end
end

{:module, M, <<70, 79, 82, 49, 0, 0, 3, 240, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 121, 0, 0, 0, 14, 8, 69, 108, 105, 120, 105, 114, 46, 77, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, 99, ...>>, {:f, 1}}

## Summary